IMPORTAÇÃO DE MODULOS

In [52]:
from config_path import *
import pandas as pd
import numpy as np
import glob
import os

EXTRAÇÃO

In [53]:
def director(files, argumento):
    director = glob.glob(os.path.join(files, argumento))
    lista = []
    for arquivo in director:
        x = pd.read_csv(arquivo, header= None)
        lista.append(x)
    df_temp = pd.concat(lista, axis= 0, ignore_index= True)
    return df_temp

df_sug = director(directory.dir_82, 'DEP*.txt')
df_mov = director(directory.dir_82, 'MOV*.txt')
df_prod = pd.read_excel(relatorios.rel_96, usecols=['CODPROD','DESCRICAO', 'QTUNITCX', 'QTTOTPAL', 'OBS2','RUA', 'PREDIO', 'APTO'])
df_acesso = pd.read_excel(relatorios.rel_60, usecols= ['CODPROD','QTOS', 'QT'])
df_estoque = pd.read_excel(outros.ou_86, usecols=['Código', 'Estoque', 'Custo ult. ent.','Qtde Pedida','Bloqueado(Qt.Bloq.-Qt.Avaria)','Qt.Avaria'])
print("\nleitura dos DATAFRAMES finalizado...\n")



c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



leitura dos DATAFRAMES finalizado...



CORPO DO CODIGO

In [ ]:
df_sug.columns = col_names.col_sug
df_sug = df_sug.drop_duplicates(subset=['COD'], keep='first')
df_sug.replace([np.inf, -np.inf], 0, inplace=True) 

df_mov.columns = col_names.col_mov
df_mov = df_mov.drop_duplicates(subset=['COD'], keep='first')
df_mov.replace([np.inf, -np.inf], 0, inplace=True) 

df_prod = df_prod.loc[df_prod['RUA'].between(1,39)]
df_prod = df_prod.rename(columns={'CODPROD': 'COD'})
df_prod['OBS2'] = df_prod['OBS2'].fillna("Ativos")
df_prod["PRODUTO"] = df_prod['COD'].astype(str)

df_estoque = df_estoque.fillna(0)
rename = {
    "Qtde Pedida" : "PEDIDO_COMP"
    ,"Bloqueado(Qt.Bloq.-Qt.Avaria)" : "BLOQUEADO"
    ,"Qt.Avaria" : "AVARIA"
}
df_estoque = df_estoque.rename(columns=rename)

temp = df_acesso.groupby('CODPROD').agg(
ACESSO = ("QTOS", "sum")
).reset_index()

df = df_prod.merge(df_sug, left_on= 'COD', right_on= 'COD', how= 'left')
df = df.merge(df_estoque, left_on='COD', right_on='Código', how='left')
df = df.merge(temp, left_on='COD', right_on='CODPROD', how= 'left')
df = df.merge(df_mov, left_on='COD', right_on='COD', how="left")

concat = df['RUA_x'].astype(str) + "-" + df['PREDIO_x'].astype(str)

df[['COM FATOR', 'QTTOTPAL', 'CAP']] = df[['COM FATOR', 'QTTOTPAL','CAP']].fillna(0)
df['PL_SUG'] = (df['COM FATOR'].astype(int) / df['QTTOTPAL'].astype(int)).fillna(0).round(2)
df['PL_CAP'] = (df['CAP'].astype(int) / df['QTTOTPAL'].astype(int)).fillna(0).round(2)
df['Custo ult. ent.'] = df['Custo ult. ent.'].astype(float).fillna(0).round(2)
df['CUSTO_TT'] = (df['Estoque'].astype(float) * df['Custo ult. ent.']).fillna(0).round(2)
df['CONTAGEM'] = concat.map(concat.value_counts())
df['STATUS_PROD'] = np.where(df['CONTAGEM'] > 3, "DIV", np.where(df['CONTAGEM'] > 2,"VAL", "INT"))
df.replace([np.inf, -np.inf], 0, inplace=True) 

drop_col = ['UNID.','DESCRIÇÃO', '%_x','RUA_y', 'PREDIO_y', 'APTO_y','Código', 'ACESSO','DESCRICAO','EMBALAGEM_y', 'UNID','%_y', '%_ACUM','NIVEL','PEDIDO_COMP', 'BLOQUEADO','AVARIA']
df_final = df.drop(columns=drop_col)

dic_rename = {
    "EMBALAGEM_x" : "EMB"
    ,"RUA_x" : "RUA"
    ,"PREDIO_x" : "PREDIO"
    ,"APTO_x" : "APTO"
    ,"Custo ult. ent." : "CUSTO_ULT"
}
df_final = df_final.rename(columns=dic_rename)

nova_ordem = ['COD', 'DESC','EMB', 'OBS2', 'QTUNITCX','QTTOTPAL', 'RUA', 'PREDIO', 'APTO', 'TIPO', 'MÊS 1', 'MÊS 2', 'MÊS 3', 'CAP', '1 DIA', 'COM FATOR', 'VARIAÇÃO','Estoque', 'CUSTO_ULT','MOVI', 'CLASSE', 'PL_SUG', 'PL_CAP', 'CONTAGEM', 'STATUS_PROD', 'CUSTO_TT','PRODUTO']
df_final = df_final[nova_ordem]
print("tratamento finalizado...\n")


ratamento finalizado...



CONSULTAS

In [56]:
display(df.columns)
display(df)
 

Index(['COD', 'DESCRICAO', 'OBS2', 'QTUNITCX', 'RUA_x', 'PREDIO_x', 'APTO_x',
       'QTTOTPAL', 'PRODUTO', 'DESCRIÇÃO', 'EMBALAGEM_x', 'UNID.', 'DEP',
       'RUA_y', 'PREDIO_y', 'NIVEL', 'APTO_y', 'MÊS 1', 'MÊS 2', 'MÊS 3',
       'TIPO', 'CAP', '1 DIA', 'COM FATOR', 'VARIAÇÃO', '%_x', 'Código',
       'Estoque', 'PEDIDO_COMP', 'BLOQUEADO', 'AVARIA', 'Custo ult. ent.',
       'CODPROD', 'ACESSO', 'DESC', 'EMBALAGEM_y', 'UNID', 'MOVI', '%_y',
       '%_ACUM', 'CLASSE', 'PL_SUG', 'PL_CAP', 'CUSTO_TT', 'CONTAGEM',
       'STATUS_PROD'],
      dtype='object')

,COD,DESCRICAO,OBS2,QTUNITCX,RUA_x,PREDIO_x,APTO_x,QTTOTPAL,PRODUTO,DESCRIÇÃO,...,UNID,MOVI,%_y,%_ACUM,CLASSE,PL_SUG,PL_CAP,CUSTO_TT,CONTAGEM,STATUS_PROD
0,31901.0,UD FERRO SECO BLACK E DECKER TRAD 220V,Ativos,9.0,1.0,44.0,101.0,40.0,31901.0,UD FERRO SECO BLACK E DECKER TRAD 220V,...,UN,221.0,"0,00","97,25",C,0.15,1.00,5813.32,2,INT
1,475848.0,UD GARRAFA T INVICTA BULE 1L C/GAT PRETO,Ativos,6.0,1.0,13.0,102.0,42.0,475848.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,1251.00,2,INT
2,467934.0,"CAIXA TERMICA COLEMAN 26,5L PRETA TP PTA",Ativos,2.0,1.0,14.0,102.0,16.0,467934.0,"CAIXA TERMICA COLEMAN 26,5L PRETA TP PTA",...,UN,10.0,"0,00","99,99",C,0.75,1.25,2456.64,2,INT
3,470859.0,UD VENTILADOR CADENCE 40CM 140W 220V,Ativos,1.0,1.0,15.0,101.0,33.0,470859.0,UD VENTILADOR CADENCE 40CM 140W 220V,...,UN,94.0,"0,00","99,19",C,0.36,0.67,61674.00,2,INT
4,470868.0,"CAIXA TERMICA COLEMAN 26,5L VERDE TP VDE",Ativos,2.0,1.0,15.0,102.0,16.0,470868.0,"CAIXA TERMICA COLEMAN 26,5L VERDE TP VDE",...,UN,4.0,"0,00","100,00",C,0.75,1.00,2456.64,2,INT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,471927.0,MAIONESE LANCHERO 3KG SACHET,Ativos,6.0,9.0,42.0,101.0,60.0,471927.0,MAIONESE LANCHERO 3KG SACHET,...,UN,662.0,"0,01","90,33",B,0.40,1.20,4771.00,2,INT
10033,470627.0,SACHET CATCHUP LANCHERO 7GR,Ativos,16.0,9.0,53.0,102.0,42.0,470627.0,SACHET CATCHUP LANCHERO 7GR,...,CX,4341.0,"0,05","59,42",A,0.29,1.14,10276.00,2,INT
10034,470628.0,SACHET MAIONESE LANCHERO 7GR,Ativos,16.0,9.0,69.0,102.0,42.0,470628.0,SACHET MAIONESE LANCHERO 7GR,...,CX,2653.0,"0,03","69,65",A,0.43,0.21,5614.00,4,DIV
10035,470629.0,SACHET MOSTARDA LANCHERO 7GR,Ativos,16.0,9.0,81.0,101.0,36.0,470629.0,SACHET MOSTARDA LANCHERO 7GR,...,CX,338.0,"0,00","95,31",C,0.33,0.25,1414.00,4,DIV
